Reference:
https://qiita.com/tnozoo/items/000528b2001fe629ced4

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import pandas_datareader.data as pdr
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import mplfinance as mpf
import yfinance as yf

In [19]:
# 12か月チャート
month = 12
# チャートの基本設定
kwargs = dict(type = 'candle', style = 'yahoo') ## starsandstripes, yahoo

symbol = '^N225'
#symbol = 'USDJPY=X'
ticker = yf.Ticker(symbol)
_df = ticker.history(period='1y')
_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-09-11 00:00:00+09:00,32690.539062,32746.140625,32391.689453,32467.759766,84200000,0.0,0.0
2023-09-12 00:00:00+09:00,32629.160156,32799.691406,32486.480469,32776.371094,94000000,0.0,0.0
2023-09-13 00:00:00+09:00,32742.289062,32872.441406,32616.650391,32706.519531,98100000,0.0,0.0
2023-09-14 00:00:00+09:00,32925.539062,33244.449219,32851.238281,33168.101562,112100000,0.0,0.0
2023-09-15 00:00:00+09:00,33428.441406,33634.308594,33391.460938,33533.089844,166600000,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-04 00:00:00+09:00,38039.910156,38080.019531,36879.328125,37047.609375,144400000,0.0,0.0
2024-09-05 00:00:00+09:00,36536.250000,37099.960938,36359.230469,36657.089844,111300000,0.0,0.0
2024-09-06 00:00:00+09:00,36815.000000,36898.281250,36235.609375,36391.468750,111700000,0.0,0.0


In [20]:
# 取得したデータの次の日から26日先までが欠損値の DataFrame を作り結合する
next_day = _df.index[-1] + relativedelta(days = 1)
_df_26 = pd.DataFrame(data=np.full(shape=26 * len(_df.columns), fill_value=np.nan).reshape(26, len(_df.columns)),
        columns=_df.columns, index=pd.date_range(next_day, periods=26, freq='D'))
df = pd.concat([_df, _df_26])
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2023-09-11 00:00:00+09:00,32690.539062,32746.140625,32391.689453,32467.759766,84200000.0,0.0,0.0
2023-09-12 00:00:00+09:00,32629.160156,32799.691406,32486.480469,32776.371094,94000000.0,0.0,0.0
2023-09-13 00:00:00+09:00,32742.289062,32872.441406,32616.650391,32706.519531,98100000.0,0.0,0.0
2023-09-14 00:00:00+09:00,32925.539062,33244.449219,32851.238281,33168.101562,112100000.0,0.0,0.0
2023-09-15 00:00:00+09:00,33428.441406,33634.308594,33391.460938,33533.089844,166600000.0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-02 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-03 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-04 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-05 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def ichimoku(o, h, l, c):
    ## 当日を含めた過去26日間の最高値
    max26 = h.rolling(window=26).max()
    ## 当日を含めた過去 9日間の最高値
    max9  = h.rolling(window=9).max()
    ## 当日を含めた過去52日間の最高値
    max52 = h.rolling(window=52).max()

    ## 当日を含めた過去26日間の最安値
    min26 = l.rolling(window=26).min()
    ## 当日を含めた過去 9日間の最安値
    min9  = l.rolling(window=9).min()
    ## 当日を含めた過去52日間の最安値
    min52 = l.rolling(window=52).min()

    ## 基準線=（当日を含めた過去26日間の最高値+最安値）÷2
    kijun = (max26 + min26) / 2
    ## 転換線=（当日を含めた過去9日間の最高値+最安値）÷2
    tenkan = (max9 + min9) / 2
    ## 先行スパン1=｛（転換値+基準値）÷2｝を26日先行させて表示
    senkospan1 = (kijun + tenkan) / 2
    senkospan1 = senkospan1.shift(26)
    ## 先行スパン2=｛（当日を含めた過去52日間の最高値+最安値）÷2｝を26日先行させて表示
    senkospan2 = (max52 + min52) / 2
    senkospan2 = senkospan2.shift(26)
    ## 遅行スパン= 当日の終値を26日遅行させて表示
    chikouspan = c.shift(-26)

    return kijun, tenkan, senkospan1, senkospan2, chikouspan

In [22]:
# float 型に
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Close'] = df['Close'].astype(float)
o = df['Open']
c = df['Close']
l = df['Low']
h = df['High']

In [23]:
'''
テクニカル指標の結果を得る
'''
# 一目均衡表
kijun, tenkan, senkospan1, senkospan2, chikouspan = ichimoku(o, h, l, c)

In [17]:
l = len(df)
mid = list(df.index)[int(l / 3)]
idx_mid = list(df.index).index(mid)

'''
チャートを描く
'''
FONT_PATH = "../fonts/RictyDiminished-Regular.ttf"
fm.fontManager.addfont(FONT_PATH)

# FontPropertiesオブジェクト生成（名前の取得のため）
font_prop = fm.FontProperties(fname=FONT_PATH)
font_prop.get_name()

plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['font.size'] = 14

fig, ax = plt.subplots(figsize=(12, 5))

# プロットを作成する(ボリンジャーバンド，MACD，RSI，一目均衡表)
apds = [
    mpf.make_addplot(kijun[mid:], color='blue', alpha=0.5, width=1, label='基準線', ax=ax),
    mpf.make_addplot(tenkan[mid:], color='magenta', alpha=0.5, width=1, label='転換線', ax=ax),
    mpf.make_addplot(senkospan1[mid:], color='darkcyan', alpha=0.5, width=0.75, label='先行スパン１', ax=ax),
    mpf.make_addplot(senkospan2[mid:], color='brown', alpha=0.5, width=0.75, label='先行スパン２', ax=ax),
    mpf.make_addplot(chikouspan[mid:], color='orange', alpha=0.75, width=1, label='遅行線', ax=ax)
]

# 一目均衡表(axes=1)の先行スパン１と先行スパン２の間を塗りつぶす
y1 = senkospan1[mid:].values
y2 = senkospan2[mid:].values
fb_span1 = dict(y1=y1, y2=y2, where=y1>y2, alpha=0.1, color='darkcyan', interpolate=True)
fb_span2 = dict(y1=y1, y2=y2, where=y1<y2, alpha=0.1, color='brown', interpolate=True)
fb_span_both = [fb_span1, fb_span2]
ax.fill_between(x=range(0, len(df.index) - idx_mid), **fb_span1)
ax.fill_between(x=range(0, len(df.index) - idx_mid), **fb_span2)

# ローソク足を描く，用意したプロットを渡す
mpf.plot(
    df[mid:],
    style='yahoo',
    type='candle',
    xrotation=0,
    addplot=apds,
    ax=ax,
)

ax.set_title('%s (%s)' % (ticker.info['longName'], symbol))
ax.legend(loc='best', fontsize=9, frameon=True)
ax.grid('both')

plt.subplots_adjust(left=0.01, right=0.93, bottom=0.06, top=0.94)
plt.savefig('daily_chart_ichimoku_n225.png')

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../fonts/RictyDiminished-Regular.ttf'